In [1]:
# Activate the benchmarking environment
using Pkg
Pkg.activate(".")

# Import the necessary packages
include("SSD_Benchmark.jl")
using .SSD_Benchmark
using StableRNGs
using StateSpaceDynamics
import HiddenMarkovModels as HMMs

  Activating project at `c:\Users\zachl\OneDrive\Documents\GitHub\StateSpaceDynamics.jl\benchmarking`


In [2]:
rng = StableRNG(50)

# Test the instance struct
I1 = HMMInstance(num_states=2, num_trials=5, seq_length=100, input_dim=2, output_dim=1)

# Test the params struct
P1 = init_params(rng, I1)

# Test building the models
M1 = build_model(SSD_GLMHMM_Implem(), I1, P1)

# Test building the data
labels, X, Y, obs_seq, control_seq, seq_ends = build_data(rng, M1, I1)

# Run a benchmark
B1 = run_benchmark(SSD_GLMHMM_Implem(), M1, X, Y)

Running EM algorithm... 100%|██████████████████████████████████████████████████| Time: 0:00:00 ( 6.45 ms/it)
Running EM algorithm... 100%|██████████████████████████████████████████████████| Time: 0:00:00 ( 1.86 ms/it)
Running EM algorithm... 100%|██████████████████████████████████████████████████| Time: 0:00:00 ( 3.14 ms/it)
Running EM algorithm... 100%|██████████████████████████████████████████████████| Time: 0:00:00 ( 2.22 ms/it)
Running EM algorithm... 100%|██████████████████████████████████████████████████| Time: 0:00:00 ( 2.34 ms/it)
Running EM algorithm... 100%|██████████████████████████████████████████████████| Time: 0:00:00 ( 2.35 ms/it)
Running EM algorithm... 100%|██████████████████████████████████████████████████| Time: 0:00:00 ( 3.32 ms/it)
Running EM algorithm... 100%|██████████████████████████████████████████████████| Time: 0:00:00 ( 2.00 ms/it)
Running EM algorithm... 100%|██████████████████████████████████████████████████| Time: 0:00:00 ( 2.30 ms/it)
Running EM algorith

(time = 2.2853215e8, memory = 73635184, allocs = 176735, success = true)

In [3]:
# Test building a HiddenMarkovModels.jl implementation
M2 = build_model(HMM_GLMHMM_Implem(), I1, P1)

B2 = run_benchmark(HMM_GLMHMM_Implem(), M2, X, Y)

(time = 1.4574335e8, memory = 135069216, allocs = 182891, success = true)

In [4]:
# Test building the dynamax model
M3, dparams, dprops = build_model(DYNAMAX_GLMHMM_Implem(), I1, P1)

B3 = run_benchmark(DYNAMAX_GLMHMM_Implem(), M3, dparams, dprops, X, Y)

(time = 6.310759e8, memory = 808, allocs = 21, success = true)